In [ ]:
%load_ext autoreload
%autoreload 2

<div class="main-title">
<h1>Geospatial data</h1>
<p>Introduction<p>
</div>

Adapted: https://github.com/jorisvandenbossche/geopandas-tutorial/
Encourage you to check out the original tutorial for more examples and exercises!

## SRAI utilizes GeoPandas

SRAI utilizes GeoPandas as its main under-the-hood library.  
Most functions either accept as input, return or otherwise work with GeoDataFrames.  
It implement functionalities on top of it, but strives to make it easy to use existing GeoPandas functionalities.

## What are GeoDataFrames

GeoDataFrames consist of:
- **geometries**: the column where spatial objects are stored,
- **properites**: the rest of the columns, describing the geometries.
GeoPandas also ships with a number of different spatial operations.

In [ ]:
import geopandas as gpd
countries = gpd.read_file("data/ne_110m_admin_0_countries.zip")
countries = countries[["ISO_A3", "NAME", "CONTINENT", "POP_EST", "geometry"]]
countries.head(5)

## Visualize the geometries

We can use `.explore()` to view the plot the geometries on an interactive map (Folium / Leaflet.js) or `.plot()` to plot them on a static map (matplotlib).

In [ ]:
countries.explore()

## Pandas operations

In [ ]:
countries.geometry

In [ ]:
countries.geometry.area

In [ ]:
countries['POP_EST'].mean()

## Let's go deeper

In [ ]:
poland_gdf = countries[countries["NAME"] == "Poland"]
poland_gdf

In [ ]:
from srai.regionalizers import AdministrativeBoundaryRegionalizer

regionalizer = AdministrativeBoundaryRegionalizer(admin_level=4)
regions_gdf = regionalizer.transform(poland_gdf)
regions_gdf.head(5)

In [ ]:
regions_gdf.plot()

In [ ]:
regions_gdf.area.mean()

## Shapely objects

In [ ]:
from shapely.geometry import Point, Polygon, LineString

In [ ]:
type(regions_gdf.iloc[0].geometry)

In [ ]:
region = regions_gdf.iloc[2].geometry
region

In [ ]:
region.area

In [ ]:
region.bounds

In [ ]:
type(region.bounds)

In [ ]:
bounds = region.bounds
line = LineString(
    [(bounds[0], bounds[1]),
    (bounds[2], bounds[3]),]
)
line

In [ ]:
gpd.GeoSeries([line, region]).plot(cmap='tab10')

In [ ]:
line.within(region)

In [ ]:
line.intersects(region)

## Geospatial indexes

H3 is a geospatial indexing system using a hexagonal grid that can be (approximately) subdivided into finer and finer hexagonal grids

In [ ]:
from srai.regionalizers import geocode_to_region_gdf, H3Regionalizer
from utils import CB_SAFE_PALLETE

prague_gdf = geocode_to_region_gdf("Prague, Czech Republic")
regionized = H3Regionalizer(resolution=7).transform(prague_gdf)
m = prague_gdf.explore(color=CB_SAFE_PALLETE[1])
regionized.explore(m=m, color=CB_SAFE_PALLETE[0])

## Spatial operations on GeoDataFrames

### Spatial joins

In [ ]:
from srai.loaders import OSMOnlineLoader

loader = OSMOnlineLoader()
prague_bikes = loader.load(prague_gdf, {"amenity": "bicycle_rental"})
prague_bikes

In [ ]:
regionized.sjoin(prague_bikes)

### Buffering

In [ ]:
ax = prague_gdf.to_crs(epsg=5070).buffer(1000).to_crs(epsg=4326).plot()
prague_gdf.plot(ax=ax, color="red")